In [1]:
from dataset import *
from tools import evaluate
import STAGATE

from sklearn.metrics import adjusted_rand_score

/mnt/5280e/twang/.conda/envs/tf-STA/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
adata, n_cluster = get_dlpfc_data(1)
STAGATE.Cal_Spatial_Net(adata, rad_cutoff=150)
adata = STAGATE.train_STAGATE(adata)
adata = STAGATE.mclust_R(adata, used_obsm='STAGATE', num_cluster=n_cluster)
obs_df = adata.obs.dropna()
ARI = adjusted_rand_score(obs_df['mclust'], obs_df['cluster'])*100
print('Adjusted rand index = %.4f' %ARI)

/mnt/5280e/twang/.conda/envs/tf-STA/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


>>> dataset name: dlpfc, slice id: 151508, size: (4384, 33538), cluster: 7.
------Calculating spatial graph...
The graph contains 25698 edges, 4384 cells.
5.8618 neighbors per cell on average.
Size of Input:  (4384, 3000)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100%|██████████| 500/500 [00:47<00:00, 10.47it/s]
R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 5.4.10
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
Adjusted rand index = 51.1079


In [3]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder 
from sklearn.cluster import KMeans

label = LabelEncoder().fit_transform(adata.obs['cluster'])
pred = LabelEncoder().fit_transform([str(x) for x in KMeans(n_clusters=n_cluster).fit_predict(adata.obsm['STAGATE'])])

ari_score = metrics.adjusted_rand_score(label, pred) * 100
print('>>> %s: %.4f' % ('STAGATE', ari_score))

>>> STAGATE: 33.0595
